In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("synthetic_sales_data_shuffled.csv")

# Display the first few rows
print(df.head())

# Check for missing values
print(df.isnull().sum())

# Basic statistics
print(df.describe())

       Date  Product_ID  Quantity_Sold      Price  Promotion  Holiday
0  1/1/2023           0             31  24.041678          0        0
1  1/1/2023           1             38  68.579963          0        0
2  1/1/2023           3             40  29.110520          0        0
3  1/1/2023           2             24  36.210623          0        0
4  1/2/2023           0              1  87.394637          0        0
Date             0
Product_ID       0
Quantity_Sold    0
Price            0
Promotion        0
Holiday          0
dtype: int64
        Product_ID  Quantity_Sold        Price    Promotion      Holiday
count  1123.000000    1123.000000  1123.000000  1123.000000  1123.000000
mean      1.979519      32.278718    55.066381     0.181656     0.071238
std       1.420040      15.117477    26.323018     0.385733     0.257336
min       0.000000       0.000000    10.067294     0.000000     0.000000
25%       1.000000      21.000000    32.420226     0.000000     0.000000
50%       2.000

In [3]:
"""" *2.1 Temporal Features*
Extract useful information from the Date column:
- *Day of the week*: Demand may vary by weekday vs. weekend.
- *Month*: Seasonal trends may affect demand.
- *Year*: Long-term trends.
- *Week of the year*: Weekly patterns."""

'" *2.1 Temporal Features*\nExtract useful information from the Date column:\n- *Day of the week*: Demand may vary by weekday vs. weekend.\n- *Month*: Seasonal trends may affect demand.\n- *Year*: Long-term trends.\n- *Week of the year*: Weekly patterns.'

In [4]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Extract temporal features
df['Day_of_week'] = df['Date'].dt.dayofweek  # Monday=0, Sunday=6
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Week_of_year'] = df['Date'].dt.isocalendar().week

In [5]:
# *2.2 Lagged Features*
#Create lagged demand features to capture historical trends:
#- *Lag 7*: Demand from 7 days ago (to capture weekly patterns).
#- *Lag 30*: Demand from 30 days ago (to capture monthly patterns).

In [6]:
# Sort by Product_ID and Date
df = df.sort_values(by=['Product_ID', 'Date'])

# Create lagged features
df['Lag_7'] = df.groupby('Product_ID')['Quantity_Sold'].shift(7)
df['Lag_30'] = df.groupby('Product_ID')['Quantity_Sold'].shift(30)

In [7]:
# *2.3 Rolling Statistics*
#Create rolling statistics to capture trends and seasonality:
#- *Rolling Mean (7 days)*: Average demand over the past 7 days.
#- *Rolling Std (7 days)*: Standard deviation of demand over the past 7 days.

In [8]:
# Rolling statistics
df['Rolling_Mean_7'] = df.groupby('Product_ID')['Quantity_Sold'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df['Rolling_Std_7'] = df.groupby('Product_ID')['Quantity_Sold'].transform(lambda x: x.rolling(window=7, min_periods=1).std())

In [9]:
# *2.4 Price Change Features*
#Capture the impact of price changes on demand:
#- *Price Change*: Difference between the current price and the price from the previous day.

In [10]:
# Price change feature
df['Price_Change'] = df.groupby('Product_ID')['Price'].diff()

In [11]:
# *2.5 Promotion and Holiday Impact*
#Create interaction features to capture the combined effect of promotions and holidays:
#- *Promotion_Holiday*: Interaction between Promotion and Holiday.

In [12]:
# Interaction feature
df['Promotion_Holiday'] = df['Promotion'] * df['Holiday']

In [13]:
# Handle Missing Value
# Fill missing values
df['Lag_7'].fillna(0, inplace=True)
df['Lag_30'].fillna(0, inplace=True)
df['Rolling_Mean_7'].fillna(df['Quantity_Sold'], inplace=True)
df['Rolling_Std_7'].fillna(0, inplace=True)
df['Price_Change'].fillna(0, inplace=True)

In [14]:
# One-hot encode Day_of_week
df = pd.get_dummies(df, columns=['Day_of_week'], prefix='Day')

# Label encode Product_ID
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Product_ID'] = le.fit_transform(df['Product_ID'])

In [15]:
# Save to a new CSV file
df.to_csv("processed_sales_data.csv", index=False)